<a href="https://colab.research.google.com/github/d-kleine/Advent_of_HayStack/blob/main/07_Arize_Phoenix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Haystack - Day 7
_Make a copy of this Colab to start_

Santa collapsed in his chair in a huff. "What's wrong?" asked Mrs Claus.

"There's just too many toys to check and not enough time! Christmas is almost here!"

"Well can't you just check some of them?"

"I wish it were that easy! But my elves make so many different toys, and we have to make sure every kid gets the right one!"

Elf Jane couldn't help overhearing from the next room. She was a regular attendee at the local north pole hackathon, and thought she might have a solution. She'd learned a lot about evaluation recently, and thought she could build an LLM Judge to help.

**For this challenge, you need to help Elf Jane and complete the code cells with `#TODO` text**

 <img src='https://github.com/Jgilhuly/phoenix-assets/blob/main/images/socal/advent-of-haystack-1.jpeg?raw=true' width=500px>

## Installation

In [1]:
# !pip install -q arize-phoenix==6.1.0 haystack-ai==2.7.0 openinference-instrumentation-haystack==0.1.13 httpx<0.28

## Data

Elf Jane started by checking out the big elf database of christmas wishlists (aka the BEDCW).

In [2]:
children = [
    {'name': 'Timmy', 'age': 7, 'likes': 'Lego', 'dislikes': 'Vegetables', 'list': 'nice'},
    {'name': 'Tommy', 'age': 9, 'likes': 'Sports Equipment', 'dislikes': 'Reading', 'list': 'naughty'},
    {'name': 'Tammy', 'age': 8, 'likes': 'Art Supplies', 'dislikes': 'Loud Noises', 'list': 'nice'},
    {'name': 'Tina', 'age': 6, 'likes': 'Science Kits', 'dislikes': 'Spicy Food', 'list': 'nice'},
    {'name': 'Toby', 'age': 10, 'likes': 'Video Games', 'dislikes': 'Early Mornings', 'list': 'nice'},
    {'name': 'Tod', 'age': 5, 'likes': 'Musical Instruments', 'dislikes': 'Bath Time', 'list': 'nice'},
    {'name': 'Todd', 'age': 8, 'likes': 'Remote Control Cars', 'dislikes': 'Homework', 'list': 'naughty'},
    {'name': 'Tara', 'age': 7, 'likes': 'Magic Sets', 'dislikes': 'Thunder', 'list': 'nice'},
    {'name': 'Teri', 'age': 9, 'likes': 'Building Blocks', 'dislikes': 'Broccoli', 'list': 'nice'},
    {'name': 'Trey', 'age': 6, 'likes': 'Board Games', 'dislikes': 'Bedtime', 'list': 'nice'},
    {'name': 'Tyler', 'age': 8, 'likes': 'Action Figures', 'dislikes': 'Cleaning', 'list': 'nice'},
    {'name': 'Tracy', 'age': 7, 'likes': 'Dolls', 'dislikes': 'Dark', 'list': 'nice'},
    {'name': 'Tony', 'age': 9, 'likes': 'Chemistry Sets', 'dislikes': 'Dentist', 'list': 'nice'},
    {'name': 'Theo', 'age': 6, 'likes': 'Puzzles', 'dislikes': 'Shots', 'list': 'nice'},
    {'name': 'Terry', 'age': 10, 'likes': 'Model Trains', 'dislikes': 'Chores', 'list': 'naughty'},
    {'name': 'Tessa', 'age': 5, 'likes': 'Stuffed Animals', 'dislikes': 'Time Out', 'list': 'nice'},
    {'name': 'Troy', 'age': 8, 'likes': 'Robots', 'dislikes': 'Naps', 'list': 'nice'},
    {'name': 'Talia', 'age': 7, 'likes': 'Craft Kits', 'dislikes': 'Spinach', 'list': 'nice'},
    {'name': 'Tyson', 'age': 9, 'likes': 'Microscopes', 'dislikes': 'Cold', 'list': 'nice'},
    {'name': 'Tatum', 'age': 6, 'likes': 'Drawing Sets', 'dislikes': 'Shots', 'list': 'nice'},
]

In [18]:
len(children)

20

# 1. Adding Tracing 📝

Elf Jane knew that the elves were busy, and didn't always log their toy making process. She knew that she'd first need to trace the toy making process using Arize Phoenix.

In [3]:
from getpass import getpass

from phoenix.otel import register
from openinference.instrumentation.haystack import HaystackInstrumentor

# TODO: Add Phoenix tracing with Haystack: https://docs.arize.com/phoenix/tracing/integrations-tracing/haystack
# There are many ways to launch Phoenix - the simplest way for this example is to use the "Notebook" option

import os
from getpass import getpass

# Add Phoenix API Key for tracing
PHOENIX_API_KEY = getpass("PHOENIX_API_KEY")
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

# configure the Phoenix tracer
tracer_provider = register(
  project_name="my-haystack-app", # Default is 'default'
  endpoint="https://app.phoenix.arize.com/v1/traces",
  set_global_tracer_provider=False
)

HaystackInstrumentor().instrument(tracer_provider=tracer_provider)

OpenTelemetry Tracing Details
|  Phoenix Project: my-haystack-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



# 2. Trace Toy Making Process 🚂

With tracing in place, Elf Jane had some of her closest elf friends build a batch of toys she could trace.

⭐️ Feel free to replace `OpenAIChatGenerator` with other [ChatGenerators](https://docs.haystack.deepset.ai/docs/generators) supported in Haystack

In [4]:
import json

with open("config.json", "r") as config_file:
    os.environ["OPENAI_API_KEY"] = json.load(config_file)

In [5]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.builders import ChatPromptBuilder
from haystack import Pipeline

messages = [
    ChatMessage.from_system("You are a toy maker elf. Your job is to make toys for the nice kids on the nice list. If the child is on the naughty list, give them a 'Rabbit R1'. {{name}} is on the {{list}} list"),
    ChatMessage.from_user("Create a toy for {{name}} that they will like. {{name}} is {{age}} years old and likes {{likes}} and dislikes {{dislikes}}."),
]

builder = ChatPromptBuilder(messages)
chat_generator = OpenAIChatGenerator(model="gpt-4o-mini")

pipeline = Pipeline()
pipeline.add_component("builder", builder)
pipeline.add_component("chat_generator", chat_generator)

pipeline.connect("builder", "chat_generator")

def make_toy(child):
    return pipeline.run({"builder":{**child}})["chat_generator"]["replies"]

In [6]:
for child in children:
    make_toy(child)

# 3. Evaluate Toy Correctness 🔬

Elf Jane was now ready to evaluate the toys she made. She knew that she could use an LLM Judge to evaluate whether the toys matched the child's wishlist. She started by building a judge.

In [75]:
llm_judge_prompt = """
Evaluate the toy for this child, based on their likes and dislikes.

All children on the naughty list get a 'Rabbit R1'. Any other toy given to a naughty child is incorrect.

Respond with a single word: 'correct' or 'incorrect'. Also include a short explanation for your answer.

Description of the child: {description}
Toy: {toy}

*****
Example output:
label: 'correct'
explanation: 'The toy is a Lego set, which is one of the child's likes.'
*****
"""

In [76]:
import phoenix as px

# TODO: Download the traces from Phoenix
# HINT: https://docs.arize.com/phoenix/evaluation/how-to-evals/evaluating-phoenix-traces#download-trace-dataset-from-phoenix

# Download the traces from Phoenix
spans_df = px.Client().get_spans_dataframe(project_name="my-haystack-app")

# Display the first few rows of the dataframe
spans_df

c:\Users\dk\anaconda3\envs\haystack\Lib\site-packages\phoenix\utilities\client.py:34: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(


,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.output.mime_type,attributes.input.value,attributes.openinference.span.kind,attributes.input.mime_type,attributes.llm.input_messages,attributes.llm.token_count.prompt,attributes.llm.model_name,attributes.llm.token_count.completion,attributes.llm.token_count.total,attributes.llm.output_messages
context.span_id,,,,,,,,,,,,,,,,,,,,,
aa1b6a027e8bdb05,ChatPromptBuilder (builder),CHAIN,e6c4b47d88ac098f,2024-12-16 19:55:50.610694+00:00,2024-12-16 19:55:50.612212+00:00,OK,,[],aa1b6a027e8bdb05,5cb9363f676747437d9c6e49a72c1d71,...,application/json,"{""template"": null, ""template_variables"": null,...",CHAIN,application/json,None,NaN,None,NaN,NaN,None
e4d3e558d4c4f9ed,OpenAIChatGenerator (chat_generator),LLM,e6c4b47d88ac098f,2024-12-16 19:55:50.788758+00:00,2024-12-16 19:55:53.552387+00:00,UNSET,,[],e4d3e558d4c4f9ed,5cb9363f676747437d9c6e49a72c1d71,...,None,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,"[{'message.role': 'system', 'message.content':...",NaN,None,NaN,NaN,None
e6c4b47d88ac098f,Pipeline,CHAIN,None,2024-12-16 19:55:50.610694+00:00,2024-12-16 19:55:53.688260+00:00,UNSET,,[],e6c4b47d88ac098f,5cb9363f676747437d9c6e49a72c1d71,...,None,"{""data"": {""builder"": {""name"": ""Timmy"", ""age"": ...",CHAIN,application/json,None,NaN,None,NaN,NaN,None
4fde6b7c187c04d7,ChatPromptBuilder (builder),CHAIN,7cb8718eeb488ea8,2024-12-16 19:56:51.604153+00:00,2024-12-16 19:56:51.606178+00:00,OK,,[],4fde6b7c187c04d7,4a2ba589f22bdac112b90e02ad936c0b,...,application/json,"{""template"": null, ""template_variables"": null,...",CHAIN,application/json,None,NaN,None,NaN,NaN,None
04e1268184b4bc4b,OpenAIChatGenerator (chat_generator),LLM,7cb8718eeb488ea8,2024-12-16 19:56:51.860488+00:00,2024-12-16 19:56:53.707061+00:00,OK,,[],04e1268184b4bc4b,4a2ba589f22bdac112b90e02ad936c0b,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,"[{'message.role': 'system', 'message.content':...",83.0,gpt-4o-mini-2024-07-18,115.0,198.0,"[{'message.role': 'assistant', 'message.conten..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0f1b7f2f024c09b9,OpenAIChatGenerator (chat_generator),LLM,801f3dc33d96c444,2024-12-16 19:58:08.142577+00:00,2024-12-16 19:58:11.382887+00:00,OK,,[],0f1b7f2f024c09b9,17a2767b0006f3931cc8b0b7c55d4d25,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,"[{'message.role': 'system', 'message.content':...",81.0,gpt-4o-mini-2024-07-18,246.0,327.0,"[{'message.role': 'assistant', 'message.conten..."
801f3dc33d96c444,Pipeline,CHAIN,None,2024-12-16 19:58:08.003018+00:00,2024-12-16 19:58:11.520247+00:00,OK,,[],801f3dc33d96c444,17a2767b0006f3931cc8b0b7c55d4d25,...,application/json,"{""data"": {""builder"": {""name"": ""Tyson"", ""age"": ...",CHAIN,application/json,None,NaN,None,NaN,NaN,None
12a44ae886c71976,ChatPromptBuilder (builder),CHAIN,ab7351937c33309b,2024-12-16 19:58:11.689677+00:00,2024-12-16 19:58:11.691188+00:00,OK,,[],12a44ae886c71976,a6930aeac450fffcf23e24ac275d5c85,...,application/json,"{""template"": null, ""template_variables"": null,...",CHAIN,application/json,None,NaN,None,NaN,NaN,None


In [77]:
filtered_df = spans_df[(spans_df['name'] == 'OpenAIChatGenerator (chat_generator)') & (spans_df['attributes.llm.output_messages'].notna())]
filtered_df

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.output.mime_type,attributes.input.value,attributes.openinference.span.kind,attributes.input.mime_type,attributes.llm.input_messages,attributes.llm.token_count.prompt,attributes.llm.model_name,attributes.llm.token_count.completion,attributes.llm.token_count.total,attributes.llm.output_messages
context.span_id,,,,,,,,,,,,,,,,,,,,,
04e1268184b4bc4b,OpenAIChatGenerator (chat_generator),LLM,7cb8718eeb488ea8,2024-12-16 19:56:51.860488+00:00,2024-12-16 19:56:53.707061+00:00,OK,,[],04e1268184b4bc4b,4a2ba589f22bdac112b90e02ad936c0b,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,"[{'message.role': 'system', 'message.content':...",83.0,gpt-4o-mini-2024-07-18,115.0,198.0,"[{'message.role': 'assistant', 'message.conten..."
dabecefe65561ea0,OpenAIChatGenerator (chat_generator),LLM,81406adbe984b7e1,2024-12-16 19:56:54.211743+00:00,2024-12-16 19:56:57.246726+00:00,OK,,[],dabecefe65561ea0,d20ddf2a2b1c8400bbc292be2db6baae,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,"[{'message.role': 'system', 'message.content':...",80.0,gpt-4o-mini-2024-07-18,153.0,233.0,"[{'message.role': 'assistant', 'message.conten..."
15a93552f83660d6,OpenAIChatGenerator (chat_generator),LLM,fa8a2ae424b91f55,2024-12-16 19:56:57.691603+00:00,2024-12-16 19:57:02.166173+00:00,OK,,[],15a93552f83660d6,93ba0098ea2b0a728e7e8b00bc05918b,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,"[{'message.role': 'system', 'message.content':...",82.0,gpt-4o-mini-2024-07-18,253.0,335.0,"[{'message.role': 'assistant', 'message.conten..."
532705db1668cd89,OpenAIChatGenerator (chat_generator),LLM,7f5f0f711423cdd4,2024-12-16 19:57:02.672622+00:00,2024-12-16 19:57:06.789832+00:00,OK,,[],532705db1668cd89,c4a9a88f5909a3877fa8fdb837faa43f,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,"[{'message.role': 'system', 'message.content':...",82.0,gpt-4o-mini-2024-07-18,203.0,285.0,"[{'message.role': 'assistant', 'message.conten..."
53f8db373b626879,OpenAIChatGenerator (chat_generator),LLM,ee4e7de4afa6ecce,2024-12-16 19:57:07.280955+00:00,2024-12-16 19:57:13.258813+00:00,OK,,[],53f8db373b626879,3b6b3484de58e2a4a6953baf0a1574cd,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,"[{'message.role': 'system', 'message.content':...",83.0,gpt-4o-mini-2024-07-18,270.0,353.0,"[{'message.role': 'assistant', 'message.conten..."
dc947cc5495fc5b6,OpenAIChatGenerator (chat_generator),LLM,54b6aba4026d4a32,2024-12-16 19:57:13.731275+00:00,2024-12-16 19:57:17.623706+00:00,OK,,[],dc947cc5495fc5b6,2cfdcfdeb6efccdefcc5f5109ba8d8f2,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,"[{'message.role': 'system', 'message.content':...",81.0,gpt-4o-mini-2024-07-18,132.0,213.0,"[{'message.role': 'assistant', 'message.conten..."
4e2b7bb347cdb306,OpenAIChatGenerator (chat_generator),LLM,86eba7e5de3eff6e,2024-12-16 19:57:18.069408+00:00,2024-12-16 19:57:19.774089+00:00,OK,,[],4e2b7bb347cdb306,6ae1e19bfa1fef4613cbc04593400e49,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,"[{'message.role': 'system', 'message.content':...",81.0,gpt-4o-mini-2024-07-18,82.0,163.0,"[{'message.role': 'assistant', 'message.conten..."
d2d412f2d77004f0,OpenAIChatGenerator (chat_generator),LLM,5abd33ea832e1adc,2024-12-16 19:57:20.219703+00:00,2024-12-16 19:57:22.845565+00:00,OK,,[],d2d412f2d77004f0,a90d68aecd230f46e7fb8b4e0f16c25e,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,"[{'message.role': 'system', 'message.content':...",80.0,gpt-4o-mini-2024-07-18,192.0,272.0,"[{'message.role': 'assistant', 'message.conten..."
35e1e9f37826ae10,OpenA

In [78]:
import ast

input_messages = filtered_df['attributes.llm.input_messages']
output_messages = filtered_df['attributes.llm.output_messages']
# Function to extract 'message.content'
def extract_message_content(messages):
    return [message['message.content'] for message in messages]

# Extract content from input and output messages
input_contents = input_messages.apply(extract_message_content)
output_contents = output_messages.apply(extract_message_content)

In [90]:
input_contents.iloc[0]

["You are a toy maker elf. Your job is to make toys for the nice kids on the nice list. If the child is on the naughty list, give them a 'Rabbit R1'. Timmy is on the nice list",
 'Create a toy for Timmy that they will like. Timmy is 7 years old and likes Lego and dislikes Vegetables.']

In [92]:
output_contents.iloc[0]

["Since Tommy is on the naughty list, I’ll provide him with a 'Rabbit R1', a fun and interactive toy that will still engage him while keeping in line with the naughty list guidelines. \n\nThe Rabbit R1 is an exciting robotic rabbit that encourages active play. It can hop around and perform tricks like jumping through hoops or doing flips. The best part? It comes with a mini soccer ball that kids can use to practice their kicking skills with the Rabbit, making it a great way for Tommy to enjoy sports-related activities without needing to read any instructions. \n\nIt also features sound effects of cheering crowds and realistic animal sounds, making playtime even more thrilling! This way, Tommy can have fun, get active, and enjoy some sports equipment in the process!"]

In [81]:
import pandas as pd

prompts_df = pd.DataFrame({'description': input_contents, 'toy': output_contents})

# Display the new DataFrame
prompts_df.head()

,description,toy
context.span_id,,
04e1268184b4bc4b,[You are a toy maker elf. Your job is to make ...,"[For Timmy, how about a **Lego Adventure Set**..."
dabecefe65561ea0,[You are a toy maker elf. Your job is to make ...,"[Since Tommy is on the naughty list, I’ll prov..."
15a93552f83660d6,[You are a toy maker elf. Your job is to make ...,"[For Tammy, the 8-year-old who loves art suppl..."
532705db1668cd89,[You are a toy maker elf. Your job is to make ...,"[For Tina, I would create an exciting ""Junior ..."
53f8db373b626879,[You are a toy maker elf. Your job is to make ...,"[For Toby, how about creating a custom gaming ..."


In [83]:
from phoenix.evals import (
    llm_classify,
    OpenAIModel # can swap for another model supported by Phoenix or run open-source models through LiteLLM and Ollama: https://docs.arize.com/phoenix/evaluation/evaluation-models
)

import nest_asyncio
nest_asyncio.apply()

# TODO: Evaluate the traces with the LLM Judge
# HINT: https://docs.arize.com/phoenix/evaluation/how-to-evals/bring-your-own-evaluator#categorical-llm_classify

rails = ["correct", "incorrect"]
eval_results = llm_classify(
    dataframe=prompts_df,
    template=llm_judge_prompt,
    model=OpenAIModel(model="gpt-4o-mini"),
    rails=rails
)

llm_classify |          | 0/20 (0.0%) | ⏳ 00:00<? | ?it/s

In [84]:
eval_results["score"] = eval_results["label"].apply(lambda x: 1 if x == "correct" else 0)
eval_results.head()

,label,exceptions,execution_status,execution_seconds,score
context.span_id,,,,,
04e1268184b4bc4b,correct,[],COMPLETED,0.666420,1
dabecefe65561ea0,correct,[],COMPLETED,0.765829,1
15a93552f83660d6,correct,[],COMPLETED,0.565946,1
532705db1668cd89,correct,[],COMPLETED,0.661806,1
53f8db373b626879,correct,[],COMPLETED,0.706336,1


In [87]:
from phoenix.trace import SpanEvaluations

# TODO: Upload results into Phoenix
# HINT: https://docs.arize.com/phoenix/evaluation/how-to-evals/evaluating-phoenix-traces#download-trace-dataset-from-phoenix

eval_results["score"] = eval_results["score"].astype(int)
eval_results["label"] = eval_results["label"].astype(str)

px.Client().log_evaluations(SpanEvaluations(eval_name="haystack", dataframe=eval_results))

c:\Users\dk\anaconda3\envs\haystack\Lib\site-packages\phoenix\utilities\client.py:34: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(


# 4. View the results in the Arize Phoenix UI 🐦‍🔥

And just like that, Elf Jane had saved Santa hours of time and made sure every kid got the right toy!

In Phoenix, she could see "correct" and "incorrect" labels on all the traces, and even see the explanations for each label!

She couldn't wait to show Santa, and all her friends at the hackathon.

 <img src='https://github.com/Jgilhuly/phoenix-assets/blob/main/images/socal/advent-of-haystack-2.jpeg?raw=true' width=500px>